In [1]:
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
import torch
from glob import glob
import seaborn as sns
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm
import pickle
import random
import pandas as pd
import itertools

import deep_snow.models
import deep_snow.dataset

In [2]:
# set random seeds 
torch.manual_seed(0)
torch.cuda.manual_seed(0)
random.seed(0)
np.random.seed(0)

In [3]:
# get paths to data
train_data_dir = '/mnt/Backups/gbrench/repos/deep-snow/data/subsets_v4/train'
train_path_list = glob(f'{train_data_dir}/ASO_50M_SD*.nc')

val_data_dir = '/mnt/Backups/gbrench/repos/deep-snow/data/subsets_v4/val'
val_path_list = glob(f'{val_data_dir}/ASO_50M_SD*.nc')

In [4]:
# select geographically representative subset of total data
from collections import defaultdict

# Function to extract basin and tile from the path
def extract_acquisition_tile(path):
    parts = path.split('_')
    basin = parts[4]
    tile = parts[7]
    return basin, tile

def select_paths(path_list, n):
    # Group paths by basin and tile
    grouped_paths = defaultdict(lambda: defaultdict(list))
    for path in path_list:
        basin, tile = extract_acquisition_tile(path)
        grouped_paths[basin][tile].append(path)
    
    # Randomly select n samples from each tile for each basin
    subsampled_paths = []
    for basin, tiles in grouped_paths.items():
        for tile, group in tiles.items():
            subsampled_paths.extend(random.sample(group, min(n, len(group))))
    return subsampled_paths

In [5]:
# define data to be returned by dataloader
selected_channels = [
    # ASO products
    'aso_sd', # ASO lidar snow depth (target dataset)
    'aso_gap_map', # gaps in ASO data
    
    # Sentinel-1 products
    'snowon_vv', # snow on Sentinel-1 VV polarization backscatter in dB, closest acquisition to ASO acquisition
    'snowon_vh', # snow on Sentinel-1 VH polarization backscatter in dB, closest acquisition to ASO acquisition
    'snowoff_vv', # snow off Sentinel-1 VV polarization backscatter in dB, closest acquisition to ASO acquisition
    'snowoff_vh', # snow off Sentinel-1 VH polarization backscatter in dB, closest acquisition to ASO acquisition
    'snowon_vv_mean', # snow on Sentinel-1 VV polarization backscatter in dB, mean of acquisition in 4 week period around ASO acquisition
    'snowon_vh_mean', # snow on Sentinel-1 VH polarization backscatter in dB, mean of acquisition in 4 week period around ASO acquisition
    'snowoff_vv_mean', # snow off Sentinel-1 VV polarization backscatter in dB, mean of acquisition in 4 week period around ASO acquisition
    'snowoff_vh_mean', # snow off Sentinel-1 VH polarization backscatter in dB, mean of acquisition in 4 week period around ASO acquisition
    'snowon_cr', # cross ratio, snowon_vh - snowon_vv
    'snowoff_cr', # cross ratio, snowoff_vh - snowoff_vv
    'delta_cr', # change in cross ratio, snowon_cr - snowoff_cr
    'rtc_gap_map', # gaps in Sentinel-1 data
    'rtc_mean_gap_map', # gaps in Sentinel-1 mean data
    
    # Sentinel-2 products 
    'aerosol_optical_thickness', # snow on Sentinel-2 aerosol optical thickness band 
    'coastal_aerosol', # snow on Sentinel-2 coastal aerosol band
    'blue', # snow on Sentinel-2 blue band
    'green', # snow on Sentinel-2 green band
    'red', # snow on Sentinel-2 red band
    'red_edge1', # snow on Sentinel-2 red edge 1 band
    'red_edge2', # snow on Sentinel-2 red edge 2 band
    'red_edge3', # snow on Sentinel-2 red edge 3 band
    'nir', # snow on Sentinel-2 near infrared band
    'water_vapor', # snow on Sentinel-2 water vapor
    'swir1', # snow on Sentinel-2 shortwave infrared band 1
    'swir2', # snow on Sentinel-2 shortwave infrared band 2
    'scene_class_map', # snow on Sentinel-2 scene classification product
    'water_vapor_product', # snow on Sentinel-2 water vapor product
    'ndvi', # Normalized Difference Vegetation Index from Sentinel-2
    'ndsi', # Normalized Difference Snow Index from Sentinel-2
    'ndwi', # Normalized Difference Water Index from Sentinel-2
    's2_gap_map', # gaps in Sentinel-2 data

    # PROBA-V global land cover dataset (Buchhorn et al., 2020)
    'fcf', # fractional forest cover
    
    # COP30 digital elevation model      
    'elevation',
    'slope',
    'aspect',
    'curvature',
    'tpi',
    'tri',

    # latitude and longitude
    'latitude',
    'longitude',

    # day of water year
    'dowy'
                    ]

In [6]:
def train_model(input_channels, epochs, n_layers):
    model = deep_snow.models.ResDepth(n_input_channels=len(input_channels), depth=n_layers)
    model.to('cuda');  # Run on GPU
    # Define optimizer and loss function
    optimizer = optim.Adam(model.parameters(), lr=0.0004)
    loss_fn = nn.MSELoss()
    epochs = epochs
    
    train_loss = []
    val_loss = []
    counter = 0
    
    # training and validation loop
    for epoch in range(epochs):
        print(f'\nStarting epoch {epoch+1}')
        epoch_loss = []
        val_temp_loss = []
    
        # Loop through training data with tqdm progress bar
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", unit="batch", ncols=130)
        for data_tuple in pbar:
            model.train()
            optimizer.zero_grad()
    
            # read data into dictionary
            data_dict = {name: tensor for name, tensor in zip(selected_channels, data_tuple)}
            # prepare inputs by concatenating along channel dimension
            inputs = torch.cat([data_dict[channel] for channel in input_channels], dim=1).to('cuda')
    
            # generate prediction
            pred_sd = model(inputs)
    
            # Limit prediction to areas with valid data
            pred_sd = torch.where(data_dict['aso_gap_map'].to('cuda') + data_dict['rtc_gap_map'].to('cuda') + data_dict['s2_gap_map'].to('cuda') == 0, pred_sd, torch.zeros_like(pred_sd).to('cuda'))
            aso_sd = torch.where(data_dict['aso_gap_map'].to('cuda') + data_dict['rtc_gap_map'].to('cuda') + data_dict['s2_gap_map'].to('cuda') == 0, data_dict['aso_sd'].to('cuda'), torch.zeros_like(pred_sd).to('cuda'))
    
            # Calculate loss
            loss = loss_fn(pred_sd, aso_sd.to('cuda'))
            epoch_loss.append(loss.item())
    
            # Update tqdm progress bar with batch loss
            pbar.set_postfix({'batch loss': loss.item(), 'mean epoch loss': np.mean(epoch_loss)})
    
    
            loss.backward()  # Propagate the gradients in backward pass
            optimizer.step()
    
        train_loss.append(np.mean(epoch_loss))
        print(f'Training loss: {np.mean(epoch_loss)}')
    
        # Run model on validation data with tqdm progress bar
        for data_tuple in tqdm(val_loader, desc="Validation", unit="batch"):
            with torch.no_grad():
                model.eval()
                
                # read data into dictionary
                data_dict = {name: tensor for name, tensor in zip(selected_channels, data_tuple)}
                # prepare inputs by concatenating along channel dimension
                inputs = torch.cat([data_dict[channel] for channel in input_channels], dim=1).to('cuda')
        
                # generate prediction
                pred_sd = model(inputs)
        
                # Limit prediction to areas with valid data
                pred_sd = torch.where(data_dict['aso_gap_map'].to('cuda') + data_dict['rtc_gap_map'].to('cuda') + data_dict['s2_gap_map'].to('cuda') == 0, pred_sd, torch.zeros_like(pred_sd).to('cuda'))
                aso_sd = torch.where(data_dict['aso_gap_map'].to('cuda') + data_dict['rtc_gap_map'].to('cuda') + data_dict['s2_gap_map'].to('cuda') == 0, data_dict['aso_sd'].to('cuda'), torch.zeros_like(pred_sd).to('cuda'))
        
                # Calculate loss
                loss = loss_fn(pred_sd, aso_sd.to('cuda'))
                val_temp_loss.append(loss.item())

        # decay loss
        if epoch == 30:
            optimizer = optim.Adam(model.parameters(), lr=0.0002)
        if epoch == 60:
            optimizer = optim.Adam(model.parameters(), lr=0.0001)
        if epoch == 90:
            optimizer = optim.Adam(model.parameters(), lr=0.00005)

        # # calculate loss over previous 10 epochs for early stopping later
        # if epoch > 20:
        #     past_loss = np.median(val_loss[-20:-10])
    
        val_loss.append(np.mean(val_temp_loss))
        print(f'Validation loss: {np.mean(val_temp_loss)}')

        # # implement early stopping
        # if epoch > 20:
        #     current_loss = np.median(val_loss[-10:-1])
        #     if current_loss > past_loss:
        #         counter +=1
        #         if counter >= 10:
        #             print('early stopping triggered')
        #             break
        #     else:
        #         counter = 0

    return np.median(val_loss[-10:])

In [7]:
# define input channels for model
all_input_channels = [
    'snowon_vv',
    'snowon_vh',
    'snowoff_vv',
    'snowoff_vh',
    'snowon_cr',
    'snowoff_cr',
    'delta_cr',
    'blue',
    'green',
    'red',
    'nir',
    'swir1',
    'swir2',
    'scene_class_map',
    'ndvi',
    'ndsi',
    'ndwi',
    'fcf',
    'elevation',
    'slope',
    'aspect',
    'curvature',
    'tri',
    'tpi',
    'latitude',
    'longitude',
    'dowy']

In [8]:
# # subsample dataset
# train_path_list = select_paths(train_path_list, 4)
# val_path_list = select_paths(val_path_list, 6)

# subsample dataset
train_path_list = random.sample(train_path_list, 1024)
val_path_list = random.sample(val_path_list, 512)

# prepare training and validation dataloaders
train_data = deep_snow.dataset.Dataset(train_path_list, selected_channels, norm=True)
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=32, shuffle=True, num_workers=16)
val_data = deep_snow.dataset.Dataset(val_path_list, selected_channels, norm=True)
val_loader = torch.utils.data.DataLoader(dataset=val_data, batch_size=32, shuffle=True, num_workers=16)

In [9]:
n_epochs = 100
n_layers = 5
experiment_data = {}

for i in range(len(all_input_channels)):
    print('---------------------------------------------------------')
    print(f'starting round {i}')
    if i == 0:
        print(f'round 1: establish baseline with all channels')
        baseline_loss = train_model(all_input_channels, epochs=n_epochs, n_layers=n_layers)
        experiment_data['r0_baseline'] = baseline_loss
        print(f'all channels baseline loss: {baseline_loss}')
        experiment_channels = all_input_channels
    else: 
        # new dictionary to store loss for the new round
        print(f'channels to ablate in round: {experiment_channels}')
        experiment_data[f'r{i}'] = {}
        new_experiment_channels = []
        # strop if only a single trial showed improvement in the last round
        if len(experiment_channels) < i:
            print('single trial showed improved loss over last round, search completed')
            baseline_channels = min(experiment_data[f'r{i-1}'], key=experiment_data[f'r{i-1}'].get)
            print(f'final features to be removed: {baseline_channels}')
            break
        else:
            # get combinations of number i of feature targeted for ablation
            channel_combinations = list(itertools.combinations(experiment_channels, i))
            for trial, trial_channels in enumerate(channel_combinations):
                print(f'starting trial {trial+1}/{len(channel_combinations)}, channels to ablate: {trial_channels}')
                # select all features except those to be held out
                input_channels = [channel for channel in experiment_channels if channel not in trial_channels]
                final_val_loss = train_model(input_channels, epochs=n_epochs, n_layers=n_layers)
                experiment_data[f'r{i}'][trial_channels] = final_val_loss
                print(f'{trial_channels} trial loss: {final_val_loss}, R0 baseline loss: {experiment_data["r0_baseline"]}')
                # update for next round if loss improves
                if final_val_loss < baseline_loss:
                    for channel in trial_channels:
                        if channel not in new_experiment_channels:
                            new_experiment_channels.append(channel)
                # save to disk
                with open(f'../../loss/ResDepth_ablation_v0.pkl', 'wb') as f:
                    pickle.dump(experiment_data, f)
            # break if no combinations show improved loss over last round
            if len(new_experiment_channels) == 0:
                print('no improved loss over last round, search completed')
                baseline_channels = min(experiment_data[f'r{i-1}'], key=experiment_data[f'r{i-1}'].get)
                print(f'final features to be removed: {baseline_channels}')
                break
            else:
                # update channels to be ablated
                experiment_channels = new_experiment_channels
                # find new baseline loss
                baseline_channels = min(experiment_data[f'r{i}'], key=experiment_data[f'r{i}'].get)
                baseline_loss = experiment_data[f'r{i}'][baseline_channels]
                print(f'new baseline: {baseline_channels}, val loss: {baseline_loss}')

---------------------------------------------------------
starting round 0
round 1: establish baseline with all channels

Starting epoch 1


Epoch 1/100: 100%|████████████████████████████████| 32/32 [00:20<00:00,  1.55batch/s, batch loss=0.00116, mean epoch loss=0.00471]


Training loss: 0.004708794233010849


Validation: 100%|████████████████████████████████████████████████████████████████████| 16/16 [00:10<00:00,  1.51batch/s]


Validation loss: 0.0010889227596635465

Starting epoch 2


...

Validation: 100%|████████████████████████████████████████████████████████████████████| 16/16 [00:10<00:00,  1.51batch/s]


Validation loss: 0.0007720444664300885

Starting epoch 20


Epoch 20/100: 100%|█████████████████████████████| 32/32 [00:20<00:00,  1.58batch/s, batch loss=0.000533, mean epoch loss=0.000803]


Training loss: 0.0008030694807530381


Validation:  62%|██████████████████████████████████████████▌                         | 10/16 [00:10<00:06,  1.03s/batch]

KeyboardInterrupt



In [ ]:
experiment_data